In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
val_df = pd.read_csv('validation.csv')

# Separate inputs (articles) and targets (summaries)
X_train, y_train = train_df['article'], train_df['highlights']
X_val, y_val = val_df['article'], val_df['highlights']
X_test, y_test = test_df['article'], test_df['highlights']

In [2]:
from sklearn.utils import shuffle
train_df_shuffled = shuffle(train_df, random_state=42)  # random_state for reproducibility
subset_size = int(0.1 * len(train_df_shuffled)) # take 0.25 of data length 
train_df = train_df_shuffled.iloc[:subset_size]
train_df

,id,article,highlights
272581,ed0fed726929c1eeabe6c390e47128dbb7d7a055,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...
772,023cd84001b33aed4ff0f3f5ecb0fdd2151cf543,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
171868,6a70a0d8d3ed365fe1df6d35f1587a8b9b298618,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
63167,b37204c13ea38b511265e41ac69fb12acfb63f85,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...
68522,c24e5805afd5145bc48410e876db91d44a06be5e,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...
...,...,...,...
225109,af82d43ecb0fb4ce0bb23e4fda724596181c6709,An ax wielding teen was left crying for his mo...,Store clerk Ali Ait Mahdi said he defended him...
73267,cfca4c7a8eff7f6c1230f0b2866504f1513c68c0,A food supplier falsely marketed beef to Musli...,"Midamar Corp, of Cedar Rapids, Iowa, sold food..."
9840,1be8e21ffcabec519f305f35643821670b485181,New York (CNN) -- Diplomacy hasn't worked. San...,Hafez Nazeri and his father Shahrem play hybri...
210557,9cb1380d06a63402cb38089b4a125184e831bbde,(CNN) -- Is Gail Kelly the most powerful woman...,"Meet Gail Kelly, CEO of Australia's second lar..."


In [3]:
train_df.drop_duplicates(['highlights'], inplace=True)
test_df.drop_duplicates(['highlights'], inplace=True)
train_df = train_df.drop(['id'], axis= 1)
train_df.reset_index(inplace=True, drop=True)
test_df = test_df.drop(['id'], axis= 1)
test_df.reset_index(inplace=True, drop=True)

C:\Users\acer\AppData\Local\Temp\ipykernel_10832\2273873252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop_duplicates(['highlights'], inplace=True)


In [4]:
train_df

,article,highlights
0,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...
1,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
2,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
3,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...
4,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...
...,...,...
28647,An ax wielding teen was left crying for his mo...,Store clerk Ali Ait Mahdi said he defended him...
28648,A food supplier falsely marketed beef to Musli...,"Midamar Corp, of Cedar Rapids, Iowa, sold food..."
28649,New York (CNN) -- Diplomacy hasn't worked. San...,Hafez Nazeri and his father Shahrem play hybri...
28650,(CNN) -- Is Gail Kelly the most powerful woman...,"Meet Gail Kelly, CEO of Australia's second lar..."


In [5]:
test_df

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...
11444,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11445,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11446,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11447,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    #convert text to doc
    doc = nlp(text)

    # filter out stopwords and punctuation
    cleaned_words = [token.text.lower()
                     for token in doc
                     if not token.is_stop and not token.is_punct]

    return " ".join(cleaned_words)


In [7]:
new_train_df = train_df[:2000]
new_train_df

,article,highlights
0,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...
1,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
2,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
3,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...
4,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...
...,...,...
1995,By . Nicola Harley . A legal firm that made mo...,Insult came in a training manual Raleys Solici...
1996,The average interest rate on an easy-access IS...,It is the lowest average rate since MoneyFacts...
1997,(CNN) -- There are plenty of reasons to fall i...,Staircases are important elements in home deco...
1998,"By . Steve Robson . PUBLISHED: . 01:29 EST, 25...",Victor Ponta says he is 'rather perplexed' by ...


In [9]:
new_train_df["clean_article"] = new_train_df['article'].apply(clean_text)
new_train_df["clean_highlights"] = new_train_df['highlights'].apply(clean_text)
new_train_df

C:\Users\acer\AppData\Local\Temp\ipykernel_10832\3094998501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df["clean_article"] = new_train_df['article'].apply(clean_text)
C:\Users\acer\AppData\Local\Temp\ipykernel_10832\3094998501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df["clean_highlights"] = new_train_df['highlights'].apply(clean_text)


,article,highlights,clean_article,clean_highlights
0,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...,mia de graaf britons flocked beaches southern ...,people enjoyed temperatures 17c brighton beach...
1,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...,couple weighed combined 32st shamed slimming f...,couple started piling pounds birth children \n...
2,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...,video footage shows heart stopping moment 17 y...,17 year old boy suffering lacerations left han...
3,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...,istanbul turkey cnn 250 people raced syrian bo...,syrians citizens hightail turkey \n ethnic tur...
4,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...,daily mail reporter published 12:53 est 3 janu...,xue long provided helicopter helped rescue \n ...
...,...,...,...,...
1995,By . Nicola Harley . A legal firm that made mo...,Insult came in a training manual Raleys Solici...,nicola harley legal firm £ 77 million miners c...,insult came training manual raleys solicitors ...
1996,The average interest rate on an easy-access IS...,It is the lowest average rate since MoneyFacts...,average interest rate easy access isa account ...,lowest average rate moneyfacts began keeping r...
1997,(CNN) -- There are plenty of reasons to fall i...,Staircases are important elements in home deco...,cnn plenty reasons fall love staircase home co...,staircases important elements home decor famil...
1998,"By . Steve Robson . PUBLISHED: . 01:29 EST, 25...",Victor Ponta says he is 'rather perplexed' by ...,steve robson published 01:29 est 25 february 2...,victor ponta says perplexed reports romanian i...


In [10]:
new_test_df = test_df[:2000]
new_test_df

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...
1995,Francois Hollande has vowed to 'show no mercy'...,14 soldiers have been accused of abusing child...
1996,"A holidaymaker still has sickness issues, nine...",James Houlder and girlfriend Vicki Hood stayed...
1997,Jordon Ibe is poised to sign a new five-year c...,Jordon Ibe will sign a new five-year contract ...
1998,Paraded in a glass coffin and flanked by jostl...,"Iraqi officials say Izzat Ibrahim al-Douri, 72..."


In [11]:
new_test_df["clean_article"] = new_test_df['article'].apply(clean_text)
new_test_df["clean_highlights"] = new_test_df['highlights'].apply(clean_text)
new_test_df

C:\Users\acer\AppData\Local\Temp\ipykernel_10832\3622091430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df["clean_article"] = new_test_df['article'].apply(clean_text)
C:\Users\acer\AppData\Local\Temp\ipykernel_10832\3622091430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df["clean_highlights"] = new_test_df['highlights'].apply(clean_text)


,article,highlights,clean_article,clean_highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,noticed plane seats appear getting smaller sma...,experts question packed planes putting passe...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...,drunk teenage boy climbed lion enclosure zoo w...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,dougie freedman verge agreeing new year deal r...,nottingham forest close extending dougie freed...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,liverpool target neto wanted psg clubs spain b...,fiorentina goalkeeper neto linked liverpool ar...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",bruce jenner break silence hour interview dian...,tell interview reality tv star 69 air friday a...
...,...,...,...,...
1995,Francois Hollande has vowed to 'show no mercy'...,14 soldiers have been accused of abusing child...,francois hollande vowed mercy french peacekeep...,14 soldiers accused abusing children young \n ...
1996,"A holidaymaker still has sickness issues, nine...",James Houlder and girlfriend Vicki Hood stayed...,holidaymaker sickness issues months contracted...,james houlder girlfriend vicki hood stayed son...
1997,Jordon Ibe is poised to sign a new five-year c...,Jordon Ibe will sign a new five-year contract ...,jordon ibe poised sign new year contract liver...,jordon ibe sign new year contract run 2020 \n ...
1998,Paraded in a glass coffin and flanked by jostl...,"Iraqi officials say Izzat Ibrahim al-Douri, 72...",paraded glass coffin flanked jostling civilian...,iraqi officials izzat ibrahim al douri 72 died...


In [13]:
new_train_df['clean_highlights'] = new_train_df['clean_highlights'].apply(lambda x: '<Start>' + ' ' + x + ' ' + '<END>')
new_train_df

C:\Users\acer\AppData\Local\Temp\ipykernel_10832\1569651108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df['clean_highlights'] = new_train_df['clean_highlights'].apply(lambda x: '<Start>' + ' ' + x + ' ' + '<END>')


,article,highlights,clean_article,clean_highlights
0,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...,mia de graaf britons flocked beaches southern ...,<Start> <Start> people enjoyed temperatures 17...
1,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...,couple weighed combined 32st shamed slimming f...,<Start> <Start> couple started piling pounds b...
2,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...,video footage shows heart stopping moment 17 y...,<Start> <Start> 17 year old boy suffering lace...
3,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...,istanbul turkey cnn 250 people raced syrian bo...,<Start> <Start> syrians citizens hightail turk...
4,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...,daily mail reporter published 12:53 est 3 janu...,<Start> <Start> xue long provided helicopter h...
...,...,...,...,...
1995,By . Nicola Harley . A legal firm that made mo...,Insult came in a training manual Raleys Solici...,nicola harley legal firm £ 77 million miners c...,<Start> <Start> insult came training manual ra...
1996,The average interest rate on an easy-access IS...,It is the lowest average rate since MoneyFacts...,average interest rate easy access isa account ...,<Start> <Start> lowest average rate moneyfacts...
1997,(CNN) -- There are plenty of reasons to fall i...,Staircases are important elements in home deco...,cnn plenty reasons fall love staircase home co...,<Start> <Start> staircases important elements ...
1998,"By . Steve Robson . PUBLISHED: . 01:29 EST, 25...",Victor Ponta says he is 'rather perplexed' by ...,steve robson published 01:29 est 25 february 2...,<Start> <Start> victor ponta says perplexed re...


In [14]:
new_test_df['clean_highlights'] = new_test_df['clean_highlights'].apply(lambda x: '<Start>' + ' ' + x + ' ' + '<END>')
new_test_df

C:\Users\acer\AppData\Local\Temp\ipykernel_10832\1247155073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df['clean_highlights'] = new_test_df['clean_highlights'].apply(lambda x: '<Start>' + ' ' + x + ' ' + '<END>')


,article,highlights,clean_article,clean_highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,noticed plane seats appear getting smaller sma...,<Start> experts question packed planes putti...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...,<Start> drunk teenage boy climbed lion enclosu...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,dougie freedman verge agreeing new year deal r...,<Start> nottingham forest close extending doug...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,liverpool target neto wanted psg clubs spain b...,<Start> fiorentina goalkeeper neto linked live...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",bruce jenner break silence hour interview dian...,<Start> tell interview reality tv star 69 air ...
...,...,...,...,...
1995,Francois Hollande has vowed to 'show no mercy'...,14 soldiers have been accused of abusing child...,francois hollande vowed mercy french peacekeep...,<Start> 14 soldiers accused abusing children y...
1996,"A holidaymaker still has sickness issues, nine...",James Houlder and girlfriend Vicki Hood stayed...,holidaymaker sickness issues months contracted...,<Start> james houlder girlfriend vicki hood st...
1997,Jordon Ibe is poised to sign a new five-year c...,Jordon Ibe will sign a new five-year contract ...,jordon ibe poised sign new year contract liver...,<Start> jordon ibe sign new year contract run ...
1998,Paraded in a glass coffin and flanked by jostl...,"Iraqi officials say Izzat Ibrahim al-Douri, 72...",paraded glass coffin flanked jostling civilian...,<Start> iraqi officials izzat ibrahim al douri...


In [15]:
X = new_train_df['clean_article'].values
y = new_train_df['clean_highlights'].values

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [17]:
article_tokenizer = Tokenizer(num_words=2000, oov_token="<unk>")
article_tokenizer.fit_on_texts(X)
input_sequences = article_tokenizer.texts_to_sequences(X)

# View the tokenized articles
print("Word Index: ", article_tokenizer.word_index)  # The word-index mapping
print("Sequences: ", input_sequences[:3])  # The tokenized form (sequence of word indices)

Word Index:  {'<unk>': 1, 'said': 2, 'year': 3, 'people': 4, 'new': 5, 'time': 6, 'told': 7, 'years': 8, 'police': 9, 'like': 10, 'old': 11, 'mr': 12, 'left': 13, 'world': 14, 'home': 15, 'family': 16, 'day': 17, '\xa0': 18, '£': 19, '000': 20, 'right': 21, 'cnn': 22, 'found': 23, 'u': 24, 'going': 25, '1': 26, 'says': 27, 's': 28, 'man': 29, 'court': 30, 'think': 31, 'life': 32, 'president': 33, 'children': 34, 'way': 35, 'work': 36, 'house': 37, 'according': 38, 'week': 39, 'know': 40, 'city': 41, 'country': 42, 'video': 43, 'obama': 44, 'government': 45, 'long': 46, 'united': 47, 'mother': 48, 'state': 49, 'est': 50, 'team': 51, 'women': 52, 'want': 53, 'today': 54, 'good': 55, '2': 56, 'school': 57, 'called': 58, 'high': 59, 'later': 60, 'took': 61, 'including': 62, '10': 63, 'pictured': 64, 'help': 65, 'days': 66, 'death': 67, 'got': 68, '2013': 69, 'night': 70, '2012': 71, 'second': 72, 'month': 73, 'come': 74, 'group': 75, 'london': 76, 'added': 77, 'came': 78, 'away': 79, 'plac

In [18]:
highlights_tokenizer = Tokenizer(num_words=2000, oov_token="<unk>")
highlights_tokenizer.fit_on_texts(y)
output_sequences = highlights_tokenizer.texts_to_sequences(y)

# View the tokenized highlights
print("Word Index: ", highlights_tokenizer.word_index)  # The word-index mapping
print("Sequences: ", input_sequences[:3])  # The tokenized form (sequence of word indices)

Word Index:  {'<unk>': 1, 'end': 2, 'start': 3, 'says': 4, 'year': 5, 'new': 6, 'said': 7, 'old': 8, '\xa0': 9, '000': 10, 'police': 11, 'years': 12, '£': 13, '1': 14, 'found': 15, 'u': 16, 'world': 17, 'people': 18, 's': 19, 'time': 20, 'home': 21, 'man': 22, 'died': 23, 'obama': 24, '2': 25, 'president': 26, 'family': 27, 'day': 28, '10': 29, 'children': 30, 'city': 31, 'week': 32, 'mother': 33, 'claims': 34, 'court': 35, 'women': 36, 'school': 37, 'group': 38, 'million': 39, 'month': 40, 'wife': 41, 'told': 42, 'woman': 43, 'second': 44, 'death': 45, 'set': 46, 'win': 47, 'left': 48, '5': 49, 'mr': 50, 'united': 51, '3': 52, 'house': 53, '4': 54, 'work': 55, 'arrested': 56, 'father': 57, 'night': 58, 'league': 59, 'months': 60, 'west': 61, 'killed': 62, 'london': 63, 'england': 64, 'men': 65, '6': 66, 'cup': 67, '20': 68, 'like': 69, 'took': 70, 'hospital': 71, 'government': 72, 'later': 73, 'david': 74, 'life': 75, 'help': 76, 'high': 77, 'called': 78, 'days': 79, 'state': 80, 'fac

In [20]:
max_article_len = max(len(i) for i in X)
max_summary_len = max(len(i) for i in y)
encoder_input_size = len(article_tokenizer.word_index) + 1
decoder_input_size = len(highlights_tokenizer.word_index) + 1

In [21]:
# Pad the sequences
max_vocab_size_en = 2000
X_padded = pad_sequences(input_sequences, maxlen=2000)
print(X_padded[0].shape)

(2000,)


In [23]:
# Pad the sequences
max_vocab_size = 200
y_padded = pad_sequences(output_sequences, maxlen=max_summary_len)
print(y_padded[0].shape)

(1168,)
